<a href="https://colab.research.google.com/github/codeefy/Twitter_sentiment_Analysis/blob/main/Twitter_Sentiment_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install kaggle

In [ ]:
#configure the path of kaggle.json file
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

Importing Twitter sentiment dataset


In [ ]:
#Api to fetch the datset from kaggle
!kaggle datasets download -d kazanova/sentiment140

Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
sentiment140.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
#extract the compress dataset
from zipfile import ZipFile
dataset='//content/sentiment140.zip'
with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('the dataset is extracted')


the dataset is extracted


Importing the Dependecies

In [ ]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
#printing the stop word in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#loading the data from csv to pandas dataframe
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')


In [ ]:
#naming the columns name and print data again
column_names=['target','id','date','flag','user','text']
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=column_names, encoding='ISO-8859-1')



In [ ]:
twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
#cheking the rows and columns

twitter_data.shape

(1600000, 6)

In [ ]:
#counting the number of missing values in the dataset
twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [ ]:
#checking the distribution of target column
twitter_data['target'].value_counts()
#This is the sentiment140 dataset. It contains 1,600,000 tweets extracted using the twitter api . The tweets have been annotated (0 = negative, 4 = positive) and they can be used to detect sentiment .

target
0    800000
4    800000
Name: count, dtype: int64

covert the target value 4 to 1

In [ ]:
twitter_data.replace({'target':{4:1}},inplace=True)

In [ ]:
twitter_data['target'].value_counts()


target
0    800000
1    800000
Name: count, dtype: int64

0--negative tweets
1--positive tweets

**Stemming**

Stemming is the process of reducing a word to its root word
example: actor,actress,acting=act

In [ ]:
port_stem=PorterStemmer()

In [ ]:
def stemming(content):
  stemmed_content=re.sub('[^a-zA-Z]','',content) # removing the word which donot belongs to alphabets
  steemed_content=stemmed_content.lower() #covert all the content in lower case
  stemmed_content=stemmed_content.split() #split all word in the list
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')] #this line of code is preprocessing text data by stemming words and removing stopwords to prepare it for further analysis or modeling.
  stemmed_content=' '.join(stemmed_content) #this line of code is converting the list of stemmed words into a single string,
  return stemmed_content

In [ ]:
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)

In [ ]:
twitter_data.head()

,target,id,date,flag,user,text,stemmed_content
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,isupsetthathecantupdatehisfacebookbytextingita...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichanidivedmanytimesfortheballmanagedtosave...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,mywholebodyfeelsitchyandlikeitsonfir
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclassnoitsnotbehavingatallimmadwhyam...


In [ ]:
print(twitter_data['stemmed_content'])

0          switchfoothttptwitpiccomyzlawwwthatsabummeryou...
1          isupsetthathecantupdatehisfacebookbytextingita...
2          kenichanidivedmanytimesfortheballmanagedtosave...
3                       mywholebodyfeelsitchyandlikeitsonfir
4          nationwideclassnoitsnotbehavingatallimmadwhyam...
                                 ...                        
1599995           justwokeuphavingnoschoolisthebestfeelingev
1599996    thewdbcomverycooltohearoldwaltinterviewshttpbl...
1599997         areyoureadyforyourmojomakeoveraskmefordetail
1599998     happythbirthdaytomybooofallltimetupacamarushakur
1599999    happycharitytuesdaythenspccsparkscharityspeaki...
Name: stemmed_content, Length: 1600000, dtype: object


In [ ]:
print(twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [ ]:
#seprate the data and model
X=twitter_data['stemmed_content'].values
Y=twitter_data['target'].values

In [ ]:
print(X)

['switchfoothttptwitpiccomyzlawwwthatsabummeryoushouldagotdavidcarrofthirddaytodoitd'
 'isupsetthathecantupdatehisfacebookbytextingitandmightcryasaresultschooltodayalsoblah'
 'kenichanidivedmanytimesfortheballmanagedtosavetherestgooutofbound' ...
 'areyoureadyforyourmojomakeoveraskmefordetail'
 'happythbirthdaytomybooofallltimetupacamarushakur'
 'happycharitytuesdaythenspccsparkscharityspeakinguphh']


In [ ]:
print(Y)

[0 0 0 ... 1 1 1]


Splitting the data into train data test data


In [ ]:
X_train,X_test,Y_train,Y_test=train_test_split(X,Y,test_size=0.2,stratify=Y,random_state=2) #test size means 20% of the data is test data and 80 percent of the data gone for training dataset
#splitting the dataset inot train test as X,Y for input features X our model predict for Y

In [ ]:
print(X.shape,X_train.shape,X_test.shape)

(1600000,) (1280000,) (320000,)


convert the textual data to numerical data

In [ ]:

# Define the TfidfVectorizer
vectorizer = TfidfVectorizer()

# Fit and transform the training data
X_train = vectorizer.fit_transform(X_train)

# Transform the test data using the same vectorizer
X_test = vectorizer.transform(X_test)


In [ ]:
print(X_train)

  (0, 4269)	1.0
  (1, 370479)	1.0
  (2, 259879)	1.0
  (3, 544664)	1.0
  (4, 1125697)	1.0
  (5, 183085)	1.0
  (6, 655945)	1.0
  (7, 427970)	1.0
  (8, 372294)	1.0
  (9, 414372)	1.0
  (10, 453207)	1.0
  (11, 946315)	1.0
  (12, 1165285)	1.0
  (13, 334894)	1.0
  (14, 1029549)	1.0
  (15, 1251397)	1.0
  (16, 26695)	1.0
  (17, 620644)	1.0
  (18, 371056)	1.0
  (19, 166385)	1.0
  (20, 939665)	1.0
  (21, 392756)	1.0
  (22, 288392)	1.0
  (23, 202690)	1.0
  (24, 1253480)	1.0
  :	:
  (1279975, 97245)	1.0
  (1279976, 1110781)	1.0
  (1279977, 938973)	1.0
  (1279978, 783482)	1.0
  (1279979, 800936)	1.0
  (1279980, 865422)	1.0
  (1279981, 587372)	1.0
  (1279982, 952979)	1.0
  (1279983, 412834)	1.0
  (1279984, 81685)	1.0
  (1279985, 59067)	1.0
  (1279986, 1015976)	1.0
  (1279987, 777245)	1.0
  (1279988, 1218717)	1.0
  (1279989, 450973)	1.0
  (1279990, 465152)	1.0
  (1279991, 249223)	1.0
  (1279992, 1169852)	1.0
  (1279993, 1170142)	1.0
  (1279994, 75556)	1.0
  (1279995, 1106994)	1.0
  (1279996, 561113)	1

In [ ]:
print(X_test)

  (22, 137710)	1.0
  (36, 790489)	1.0
  (55, 1232076)	1.0
  (104, 806215)	1.0
  (316, 675311)	1.0
  (317, 183111)	1.0
  (326, 530903)	1.0
  (343, 483232)	1.0
  (356, 1004791)	1.0
  (405, 294046)	1.0
  (411, 754848)	1.0
  (412, 671787)	1.0
  (503, 922737)	1.0
  (575, 484366)	1.0
  (576, 68316)	1.0
  (585, 553739)	1.0
  (597, 437994)	1.0
  (704, 437478)	1.0
  (730, 522523)	1.0
  (735, 1190477)	1.0
  (763, 201540)	1.0
  (851, 230655)	1.0
  (887, 945334)	1.0
  (905, 1080771)	1.0
  (965, 64115)	1.0
  :	:
  (319388, 469686)	1.0
  (319391, 363431)	1.0
  (319401, 69391)	1.0
  (319403, 1010173)	1.0
  (319419, 435824)	1.0
  (319442, 1154007)	1.0
  (319462, 331999)	1.0
  (319474, 363237)	1.0
  (319550, 1020186)	1.0
  (319573, 1087684)	1.0
  (319595, 377143)	1.0
  (319630, 416244)	1.0
  (319632, 324587)	1.0
  (319640, 333811)	1.0
  (319649, 564793)	1.0
  (319673, 473917)	1.0
  (319679, 363451)	1.0
  (319687, 874326)	1.0
  (319700, 937697)	1.0
  (319712, 230591)	1.0
  (319779, 553486)	1.0
  (319838

training the machine learning model

Logistic Regression


In [ ]:
model=LogisticRegression(max_iter=1000)


In [ ]:
model.fit(X_train,Y_train)

LogisticRegression(max_iter=1000)

Model Evaluation


Accuracy Score

In [ ]:
#accuracy score on the training dataset
X_train_prediction=model.predict(X_train)
training_data_accuracy=accuracy_score(Y_train,X_train_prediction)

In [ ]:
print('Accuracy score on the training dataset',training_data_accuracy)

Accuracy score on the training dataset 0.99812265625


In [ ]:
#accuracy score on the training dataset
X_test_prediction=model.predict(X_test)
test_data_accuracy=accuracy_score(Y_test,X_test_prediction)

In [ ]:
print(test_data_accuracy)

0.5126375
